In [13]:
# Librerias basicas 
import numpy as np 
import pandas as pd 
import os
import re

# Visualizacióon
import matplotlib.pyplot as plt
import seaborn as sns 
from plotly import express as px

# Librerias analisis de texto
from wordcloud import WordCloud

# Funciones auxiliares sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold # Split y cross Validation
from sklearn.metrics import cohen_kappa_score, make_scorer, accuracy_score, balanced_accuracy_score, confusion_matrix, roc_curve, auc # Metricas
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict, GridSearchCV

# Models
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier as rfc
import xgboost as xgb

# Guardar modelos
import pickle

# Optimizacion de hiperparametros
import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

# Para NLP
import spacy
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from wordcloud import WordCloud
from textblob import TextBlob
import tensorflow as tf
from keybert import KeyBERT
from bertopic import BERTopic
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
from scipy import stats
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob

# Estandarizar o Normalizar
from sklearn.preprocessing import StandardScaler

# Ensemble
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

#Guardado de objetos en archivos joblib
from joblib import load, dump

%matplotlib inline
pd.set_option('display.max_columns', 30)
plt.rcParams['figure.figsize'] = [12.0, 8.0]

In [6]:
#Subimos dos niveles para quedar en la carpeta que contiene input y lab2-mcd-austral
BASE_DIR = './'

#Datos de entrenamiento 
PATH_TO_TRAIN = os.path.join(BASE_DIR, "dataset.csv")


In [8]:
df = pd.read_csv(PATH_TO_TRAIN, sep=';')
df

,Ejercicio,Tipo_Cpbte,N°_Entrada,Entidad_Nº,Cod_Ret,Fte_Fin.,Cuit/DniOtros,Clase_Registro,Clase_Gasto,Glosa,Sueldo
0,2021,AF,137,25,217,10,710,ANT,NaN,RES 7/21 ANTICIPO SUB. Y SUBVEN. CTA 360000200...,0
1,2021,AF,138,25,202,10,710,ANT,NaN,RES 8/21 ANT. VIATICOS Y MOVIL. CUENTA Nº3600...,0
2,2021,AF,150,50,229,10,30517999551,ANT,NaN,AFR-COMUNAS RURALES PLAN OBRAS Y SERVICIOS 2020,0
3,2021,AF,151,50,208,10,30517999551,ANT,NaN,"AFR-PLAN OBRAS, EQUIPAMIENTOS Y SERVICIOS 2020...",0
4,2021,AF,152,50,229,10,30517999551,ANT,NaN,"AFR-PLAN DE OBRAS, EQUIPAMIENTO Y SERVICIOS 2020",0
...,...,...,...,...,...,...,...,...,...,...,...
152218,2023,GE,30564,19,391,10,490,PAG,,LEY RI 2100-23 CTA 360000200973511 LEY 7991 CA...,1
152219,2023,GE,30565,19,391,10,490,PAG,,LEY RI 2099-23 CTA 360000200973511 LEY 7991 DG...,1
152220,2023,GE,30570,18,391,10,530,PAG,,R-3887-DGC-23-LEY 7991 NOV 2023 - CTA 36000020...,1
152221,2023,GE,30585,6,391,10,80,PAG,,RES-359-RO-LEY7991-HABERES DIC-3 CTA 789009,1


In [12]:
print(f"Filas {df.shape[0]} y Columnas {df.shape[1]}")

Filas 152223 y Columnas 11


In [9]:
df[['Glosa']].head(20)

,Glosa
0,RES 7/21 ANTICIPO SUB. Y SUBVEN. CTA 360000200...
1,RES 8/21 ANT. VIATICOS Y MOVIL. CUENTA Nº3600...
2,AFR-COMUNAS RURALES PLAN OBRAS Y SERVICIOS 2020
3,"AFR-PLAN OBRAS, EQUIPAMIENTOS Y SERVICIOS 2020..."
4,"AFR-PLAN DE OBRAS, EQUIPAMIENTO Y SERVICIOS 2020"
5,RES01- 1º ENTREGA COCINA-CTA360000200972235
6,D/130/SH/20-AFR-PAGO HABERES ENE/20-CTA 200982250
7,AFR-PAGO HABERES FEBRERO/20-CB-200981776-MI
8,AFR-PAGO HABERES FEBRERO/20-CB-200982250-MI
9,AFR-PAGO HABERES MARZO/2020 CTA.Nº200981776-MI


In [17]:
#pattern = r'^(RES|AFR|DCTO)'
pattern = r'^(RES)'
df_filtered = df[df['Glosa'].str.match(pattern)]

In [19]:
df_filtered.Glosa.head(20)

0      RES 7/21 ANTICIPO SUB. Y SUBVEN. CTA 360000200...
1      RES 8/21 ANT. VIATICOS Y MOVIL.  CUENTA Nº3600...
5            RES01- 1º ENTREGA COCINA-CTA360000200972235
104    RES 16/21 ANTICIPO DE FONDOS PNC NAC CTA 36000...
107         RES02- 1º ENTREGA BEBIDAS-CTA360000200972235
108        RES03- 1º ENTREGA CORTESIA-CTA360000200972235
109    RES09-1º ENTREGA GTOS MANT Y REP-CTA3600002009...
112    RES24 DCG-1ª ANT VIATICOS AÑO 2021 MAN JUD-CC3...
113         RES-11-RO-1ºANTICIPO VIATICO/PJES.CTA78897/2
115    RES.9/955 CTA.362200200005289 ANT. VIATICOS ENERO
134    RESOL.22(SERI)-1° ANTICIPO DE CORTESIA-CT.3600...
135    RES 34/4 CTA 817121 ANTICIPO VIATICOS SEATYDL ...
137            RES 400-CTA 817121-ANTICIPO VIATICO PG 11
139      RES N109 CTA.817121 ANTICIPO VIATICOS SEGA PG15
142    RES.34/MDP-ANT. VIATICOS N°1- CTA.360000002926...
143               RES-21-RO-1°ANTAYUDA SOCIAL-CTA78897/2
144    RES-20-RO- 1° ANTIC-CORTES Y HOMENAJE-CTA 78897/2
145    RES.30/SEPYME-ANTIC.VIAT